In [1]:
import spikeinterface.extractors as se
import spikeinterface.widgets as sw
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import pearsonr
import json
import os


In [ ]:
import os
import spikeinterface.extractors as se

# 设置路径
filePath = r"H:\elec\source"
folderName = "3-5/pre-sleep"
sourceType = "RAW"
full_path = os.path.join(filePath, folderName)

print("=== 权限和路径检查 ===")
print(f"完整路径: {full_path}")
print(f"路径存在: {os.path.exists(full_path)}")
if os.path.exists(full_path):
    print(f"可读权限: {os.access(full_path, os.R_OK)}")
    
    try:
        files = os.listdir(full_path)
        print(f"文件数量: {len(files)}")
        mpx_files = [f for f in files if f.endswith('.mpx')]
        print(f".mpx 文件: {mpx_files}")
    except Exception as e:
        print(f"列出文件失败: {e}")

print("\n=== 尝试读取 ===")
try:
    PreRecordingLFP = se.read_alphaomega(full_path, stream_id=sourceType)
    print("读取成功!")
    print(PreRecordingLFP)
except Exception as e:
    print(f"读取失败: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# 3-1 post sleep only!!!
import os
import shutil
import tempfile
# 3-1 post-sleep 21
filePath = r"H:\elec\source"
folderName = "3-1/post-sleep"
source_dir = os.path.join(filePath, folderName)

# 几个文件测试
temp_dir = tempfile.mkdtemp()
try:
    files_to_copy = ['mapfile0002.mpx', 'mapfile0003.mpx', 'mapfile0004.mpx','mapfile0005.mpx','mapfile0006.mpx','mapfile0007.mpx','mapfile0008.mpx','mapfile0009.mpx','mapfile0010.mpx','mapfile0011.mpx','mapfile0012.mpx','mapfile0013.mpx','mapfile0014.mpx','mapfile0015.mpx','mapfile0016.mpx','mapfile0017.mpx','mapfile0018.mpx','mapfile0019.mpx','mapfile0020.mpx','mapfile0021.mpx']
    
    for file in files_to_copy:
        src = os.path.join(source_dir, file)
        dst = os.path.join(temp_dir, file)
        shutil.copy2(src, dst)
    
    # 尝试读取
    PreRecordingLFP = se.read_alphaomega(temp_dir, stream_id="RAW")
    print("使用部分文件读取成功!")
    print(PreRecordingLFP)
    
except Exception as e:
    print(f"部分文件读取失败: {e}")
finally:
    shutil.rmtree(temp_dir, ignore_errors=True)

In [ ]:
## For "Alpha omega" EEG-EMG data

# 1. Setting
# [7,8] [0,15]
EEGchannelNum = [0,15]
EMGchannelNum = [7,8]
filePath = "../source"
samplingFrequency = 1875 # 后面我自己手动降采样了
folderName = "2-1/sleep"
sourceType = "RAW"
# 2. Read .mpx files
PreRecordingLFP = se.read_alphaomega(filePath+"/"+folderName, stream_id=sourceType)
print(PreRecordingLFP)

In [ ]:
# 首先检查有多少个段
num_segments = PreRecordingLFP.get_num_segments()
print(f"记录包含 {num_segments} 个段")

# 获取段信息
for i in range(num_segments):
    num_frames = PreRecordingLFP.get_num_frames(segment_index=i)
    print(f"段 {i}: {num_frames} 帧")

# 如果只有一个段，使用 segment_index=0
if num_segments == 1:
    gain = PreRecordingLFP.get_channel_gains()[0]
    if sourceType == "RAW":
        EEG_amplitude = PreRecordingLFP.get_traces(segment_index=0)[:, EEGchannelNum[0]][::16]/gain
        EMG_1_amplitude = PreRecordingLFP.get_traces(segment_index=0)[:, EMGchannelNum[0]][::16]/gain
        EMG_2_amplitude = PreRecordingLFP.get_traces(segment_index=0)[:, EMGchannelNum[1]][::16]/gain
    else:
        EEG_amplitude = (PreRecordingLFP.get_traces(segment_index=0)[:, EEGchannelNum[0]] - PreRecordingLFP.get_traces(segment_index=0)[:, EEGchannelNum[1]])/gain
        EMG_1_amplitude = PreRecordingLFP.get_traces(segment_index=0)[:, EMGchannelNum[0]]/gain
        EMG_2_amplitude = PreRecordingLFP.get_traces(segment_index=0)[:, EMGchannelNum[1]]/gain

In [ ]:
from spikeinterface import concatenate_recordings

# 如果多个段是连续的，可以连接它们
if PreRecordingLFP.get_num_segments() > 1:
    PreRecordingLFP = concatenate_recordings([PreRecordingLFP])


In [ ]:
## easy cal setup calculate ampitude and Plot

# Pre process to numpy array and show trace
gain = PreRecordingLFP.get_channel_gains()[0]
if sourceType == "RAW":
    EEG_amplitude = PreRecordingLFP.get_traces()[:, EEGchannelNum[0]][::16]/gain
    # EEG_amplitude = (PreRecordingLFP.get_traces()[:, EEGchannelNum[0]][::16]-PreRecordingLFP.get_traces()[:, EEGchannelNum[1]][::16])/gain
    EMG_1_amplitude = (PreRecordingLFP.get_traces()[:, EMGchannelNum[0]][::16])/gain
    EMG_2_amplitude = (PreRecordingLFP.get_traces()[:, EMGchannelNum[1]][::16])/gain
else:
    EEG_amplitude = (PreRecordingLFP.get_traces()[:, EEGchannelNum[0]]-PreRecordingLFP.get_traces()[:, EEGchannelNum[1]])/gain
    EMG_1_amplitude = (PreRecordingLFP.get_traces()[:, EMGchannelNum[0]])/gain
    EMG_2_amplitude = (PreRecordingLFP.get_traces()[:, EMGchannelNum[1]])/gain   
samples = len(EEG_amplitude)
totalTime = samples/samplingFrequency
timeEEG_EMG = np.linspace(0, totalTime, samples)
PreRecordingLFP = None
# PLOT
plt.figure(figsize=(40,10))
plt.subplot(2, 1, 1)
plt.plot(timeEEG_EMG, EMG_1_amplitude-EMG_2_amplitude)
plt.subplot(2, 1, 2)
plt.plot(timeEEG_EMG, EEG_amplitude)
plt.show()

In [4]:
## save EEG_EMG
if os.path.exists(filePath+"/"+folderName+'/EEG_EMG'):
    pass
else:
    os.makedirs(filePath+"/"+folderName+'/EEG_EMG')
output = {
    "EEG": EEG_amplitude.tolist(),
    "EMG_1": EMG_1_amplitude.tolist(),
    "EMG_2": EMG_2_amplitude.tolist(),
    "time": timeEEG_EMG.tolist()
}
output_json = json.dumps(output)
file = open(filePath+"/"+folderName+'/EEG_EMG/EEG_EMG_source.json', 'w')
file.write(output_json)
file.close()

In [16]:
## Re-load EEG_EMG
filePath = "../source"
folderName = "2-1/sleep"

with open(filePath+"/"+folderName+'/EEG_EMG/EEG_EMG_source.json', 'r', encoding="utf-8") as file:
    EEG_EMG_source = json.load(file)
EEG_amplitude = EEG_EMG_source["EEG"]
EMG_1_amplitude = EEG_EMG_source["EMG_1"]
EMG_2_amplitude = EEG_EMG_source["EMG_2"]
timeEEG_EMG = EEG_EMG_source["time"]

In [ ]:
## Butterworth for one channel

# 1. Setting
order = 4 #Filter order. The bigger the divide, but the steeper the transition
EEGfreqLim = [1,100] # According Method: https://doi.org/10.1038/s41586-024-08340-w
EMGfreqLim = [300,600]
samplingFrequency = 1875
# 2. Calculate
low_cutoff = EEGfreqLim[0]
high_cutoff = EEGfreqLim[1]
Wn_low = 2*low_cutoff/samplingFrequency
Wn_high = 2*high_cutoff/samplingFrequency
b, a = signal.butter(N=2, Wn=[Wn_low, Wn_high], btype='bandpass')
EEG_filtered_amplitude = signal.filtfilt(b, a, EEG_amplitude)

low_cutoff = EMGfreqLim[0]
high_cutoff = EMGfreqLim[1]
Wn_low = 2*low_cutoff/samplingFrequency
Wn_high = 2*high_cutoff/samplingFrequency
b, a = signal.butter(N=order, Wn=[Wn_low, Wn_high], btype='bandpass')
EMG_1_filtered_amplitude = signal.filtfilt(b, a, EMG_1_amplitude)
EMG_2_filtered_amplitude = signal.filtfilt(b, a, EMG_2_amplitude)

# EEG stft
window = 18750
noverlap = 18750-1875*1
maxColor = 10
mincolor = 0
minFreq = 0
maxFrea = 400
maxColor = 10
mincolor = 0
f, t, Zxx = signal.stft(EEG_filtered_amplitude, fs=samplingFrequency, window='hann', nperseg=window, noverlap=noverlap)

# 3. Plot
plt.figure(figsize=(80,20))
plt.subplot(3, 1, 1)
plt.plot(timeEEG_EMG, EMG_1_filtered_amplitude-EMG_2_filtered_amplitude)
plt.xlim([min(timeEEG_EMG), max(timeEEG_EMG)])
plt.subplot(3, 1, 2)
plt.plot(timeEEG_EMG, EEG_filtered_amplitude)
plt.xlim([min(timeEEG_EMG), max(timeEEG_EMG)])
plt.subplot(3, 1, 3)
plt.imshow(np.abs(Zxx), cmap="jet", interpolation='bicubic', vmax=maxColor, vmin=mincolor, aspect='auto')
plt.ylim([minFreq, maxFrea])
plt.show()

In [ ]:
## Sleep-state classification
# Method: https://doi.org/10.1038/s41586-024-08340-w

# Theta dominance was quantified as the ratio of powers in the 5–10-Hz and 2–16-Hz frequency bands.

# LFP PCA to one dem
pca = PCA(n_components=1)
scaler = MinMaxScaler()
LFP_reduced = np.squeeze(scaler.fit_transform(pca.fit_transform(abs(Zxx.T))))

# theta calculate
innerStartIndex = np.argmin(abs(f-5))
innerEndIndex = np.argmin(abs(f-10))
outerStartIndex = np.argmin(abs(f-2))
outerEndIndex = np.argmin(abs(f-16))
innerEnergy = np.sum(np.abs(Zxx)[innerStartIndex:innerEndIndex,:],0)
outerEnergy = np.sum(np.abs(Zxx)[outerStartIndex:outerEndIndex,:],0)
ThetaRatio = np.squeeze(scaler.fit_transform((innerEnergy/outerEnergy).reshape(-1, 1)))

# EMG zero time-lag correlation coefficients (r) at different sites and EMG one
EMG_corr = []
for i in range(0,len(EMG_1_filtered_amplitude),1875):
    EMG_1_cut = EMG_1_filtered_amplitude[i:i+1875]
    EMG_2_cut = EMG_2_filtered_amplitude[i:i+1875]
    r, p_value = pearsonr(EMG_1_cut, EMG_2_cut)
    EMG_corr.append(r)
EMG_corr.insert(0,0.5) # 长度问题注意一下，EMG少一个值，在
EMG_corr = np.squeeze(scaler.fit_transform(np.array(EMG_corr).reshape(-1, 1)))


# plot histogram
bins = np.arange(0, 1.1, 0.1)
hist_LFP, bin_edges = np.histogram(LFP_reduced, bins=bins)
hist_Theta, bin_edges = np.histogram(ThetaRatio, bins=bins)
hist_EMG, bin_edges = np.histogram(EMG_corr, bins=bins)

plt.figure(figsize=(15,5))
plt.subplot(1, 3, 1)
plt.bar(bins[0:-1], hist_LFP,width=0.08)
plt.title("PC1")
plt.subplot(1, 3, 2)
plt.bar(bins[0:-1], hist_Theta,width=0.08)
plt.title("Theta")
plt.subplot(1, 3, 3)
plt.bar(bins[0:-1], hist_EMG,width=0.08)
plt.title("EMG")

In [ ]:
# classification (REM error??)

# High LFP principal component and the low EMG were considered NREM sleep
# the high theta and low EMG were considered REM sleep
# and the remaining times were taken as WAKE.
Wake_index = []
REM_index = []
NREM_index = []

type = 0
for i in range(0,len(LFP_reduced)):
    if LFP_reduced[i]>=0.2 and EMG_corr[i]<=0.9:
        NREM_index.append(i)
        type = 1
    elif ThetaRatio[i]>=0.5 and EMG_corr[i]<=0.9:
        if type != 3:
            REM_index.append(i)
            type = 2
        else:
            Wake_index.append(i)
            type = 3
    else:
        Wake_index.append(i)
        type = 3

# 3d PLOT
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(ThetaRatio[Wake_index], LFP_reduced[Wake_index], EMG_corr[Wake_index], s=2)
ax.scatter(ThetaRatio[NREM_index], LFP_reduced[NREM_index], EMG_corr[NREM_index], s=2)
ax.scatter(ThetaRatio[REM_index], LFP_reduced[REM_index], EMG_corr[REM_index], s=2)

ax.set_xlabel('ThetaRatio')
ax.set_ylabel('PC1')
ax.set_zlabel('EMG')

In [ ]:
# plot res
Wake_index_np = np.array(Wake_index)
REM_index_np = np.array(REM_index)
NREM_index_np = np.array(NREM_index)


plt.figure(figsize=(80,20))
plt.subplot(2, 1, 1)
plt.imshow(np.abs(Zxx), cmap="jet", interpolation='bicubic', vmax=maxColor, vmin=mincolor, aspect='auto')
plt.ylim([minFreq, maxFrea])
plt.subplot(2, 1, 2)
plt.eventplot([REM_index_np, NREM_index_np, Wake_index_np], colors=['red','blue','black'])
plt.xlim([0, len(t)])
plt.show()

In [10]:
## classfication out

if os.path.exists(filePath+"/"+folderName+'/EEG_EMG'):
    pass
else:
    os.makedirs(filePath+"/"+folderName+'/EEG_EMG')
output = {
    "Wake_index": Wake_index,
    "REM_index": REM_index,
    "NREM_index": NREM_index,
    "freq": 1
}
output_json = json.dumps(output)
file = open(filePath+"/"+folderName+'/EEG_EMG/EEG_EMG_classfication.json', 'w')
file.write(output_json)
file.close()


In [ ]:
## stage&period(new)

import os
import json
import numpy as np
import matplotlib.pyplot as plt
import spikeinterface.extractors as se
from scipy import signal
import pandas as pd

In [ ]:
# 设置基础路径和当前小鼠信息
filePath = r"H:\elec\source"

current_mouse_folder = "1-2/sleep"  # 修改这里来切换不同的小鼠

EEGchannelNum = [0,15]
EMGchannelNum = [7,8]
samplingFrequency = 1875
sourceType = "RAW"

In [ ]:
# 代码块 4: 可视化分类结果（不添加剪切线）
# 首先查看完整的分类结果，以确定剪切时间点

# 加载分类结果
json_path = os.path.join(filePath, current_mouse_folder, 'EEG_EMG', 'EEG_EMG_classfication.json')
with open(json_path, 'r') as f:
    classification = json.load(f)

Wake_index = np.array(classification["Wake_index"])
REM_index = np.array(classification["REM_index"])
NREM_index = np.array(classification["NREM_index"])
freq = classification["freq"]

# 确定总样本数 - 使用最大索引值+1
total_samples = max(np.max(Wake_index) if len(Wake_index) > 0 else 0,
                   np.max(REM_index) if len(REM_index) > 0 else 0,
                   np.max(NREM_index) if len(NREM_index) > 0 else 0) + 1

print(f"总样本数: {total_samples}")
print(f"Wake索引数: {len(Wake_index)}")
print(f"REM索引数: {len(REM_index)}")
print(f"NREM索引数: {len(NREM_index)}")

# 创建与总样本数相同长度的布尔数组
Wake_bool = np.zeros(total_samples, dtype=bool)
REM_bool = np.zeros(total_samples, dtype=bool)
NREM_bool = np.zeros(total_samples, dtype=bool)

# 将索引位置设置为True
Wake_bool[Wake_index] = True
REM_bool[REM_index] = True
NREM_bool[NREM_index] = True

# 创建时间轴
time_axis = np.arange(total_samples) / freq / 3600  # 转换为小时

# 绘制分类结果
plt.figure(figsize=(15, 8))

# 创建睡眠阶段图
sleep_stages = np.zeros(total_samples)
sleep_stages[Wake_bool] = 3  # Wake
sleep_stages[NREM_bool] = 2  # NREM
sleep_stages[REM_bool] = 1   # REM

plt.subplot(2, 1, 1)
plt.plot(time_axis, sleep_stages, linewidth=2)
plt.yticks([1, 2, 3], ['REM', 'NREM', 'Wake'])
plt.ylabel('Sleep Stage')
plt.title(f'Sleep Stage Classification - {current_mouse_folder}')
plt.grid(True)

# 绘制各阶段比例
plt.subplot(2, 1, 2)
stages = ['Wake', 'NREM', 'REM']
counts = [np.sum(Wake_bool), np.sum(NREM_bool), np.sum(REM_bool)]
total = sum(counts)
percentages = [count/total*100 for count in counts]

bars = plt.bar(stages, percentages, color=['red', 'blue', 'green'])
plt.ylabel('Percentage (%)')
plt.title('Sleep Stage Distribution')

# 在柱状图上添加数值
for bar, percentage in zip(bars, percentages):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
            f'{percentage:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print(f"总记录时间: {time_axis[-1]:.2f} 小时")
print(f"Wake比例: {percentages[0]:.1f}%")
print(f"NREM比例: {percentages[1]:.1f}%")
print(f"REM比例: {percentages[2]:.1f}%")

In [ ]:
# 代码块 5: 设置剪切时间并可视化剪切线
# 基于上面的图表，确定剪切时间点（单位：小时）
start_cut_time_hours = 0.2  # 修改这个值来调整开始剪切时间
end_cut_time_hours = 1   # 修改这个值来调整结束剪切时间，设为None表示不进行结束剪切

# 计算剪切点的样本索引
start_cut_sample = int(start_cut_time_hours * 3600 * freq)
if end_cut_time_hours is not None:
    end_cut_sample = int(end_cut_time_hours * 3600 * freq)
else:
    end_cut_sample = len(Wake_bool)  # 如果不指定结束时间，则使用数据末尾

# 剪切数据
Wake_bool_trimmed = Wake_bool[start_cut_sample:end_cut_sample]
REM_bool_trimmed = REM_bool[start_cut_sample:end_cut_sample]
NREM_bool_trimmed = NREM_bool[start_cut_sample:end_cut_sample]

# 重新绘制分类结果，并添加剪切线
plt.figure(figsize=(15, 8))

plt.subplot(2, 1, 1)
# 创建剪切后的睡眠阶段图
sleep_stages_trimmed = np.zeros(len(Wake_bool_trimmed))
sleep_stages_trimmed[Wake_bool_trimmed] = 3  # Wake
sleep_stages_trimmed[NREM_bool_trimmed] = 2  # NREM
sleep_stages_trimmed[REM_bool_trimmed] = 1   # REM

time_axis_trimmed = np.arange(len(Wake_bool_trimmed)) / freq / 3600  # 转换为小时

plt.plot(time_axis_trimmed, sleep_stages_trimmed, linewidth=2)
plt.yticks([1, 2, 3], ['REM', 'NREM', 'Wake'])
plt.ylabel('Sleep Stage')
plt.title(f'Sleep Stage Classification - {current_mouse_folder} (剪切时间: {start_cut_time_hours}~{end_cut_time_hours if end_cut_time_hours else time_axis[-1]:.2f}小时)')
plt.grid(True)
plt.axvline(x=0, color='red', linestyle='--', linewidth=2, label=f'开始剪切: {start_cut_time_hours:.2f} h')
if end_cut_time_hours is not None:
    plt.axvline(x=time_axis_trimmed[-1], color='red', linestyle='--', linewidth=2, label=f'结束剪切: {end_cut_time_hours:.2f} h')
plt.legend()

# 绘制剪切后的各阶段比例
stages_trimmed = ['Wake', 'NREM', 'REM']
counts_trimmed = [np.sum(Wake_bool_trimmed), np.sum(NREM_bool_trimmed), np.sum(REM_bool_trimmed)]
total_trimmed = sum(counts_trimmed)
percentages_trimmed = [count/total_trimmed*100 for count in counts_trimmed]

plt.subplot(2, 1, 2)
bars_trimmed = plt.bar(stages_trimmed, percentages_trimmed, color=['red', 'blue', 'green'])
plt.ylabel('Percentage (%)')
plt.title('剪切后的睡眠阶段分布')

# 在柱状图上添加数值
for bar, percentage in zip(bars_trimmed, percentages_trimmed):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
            f'{percentage:.1f}%', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print(f"原始总时间: {time_axis[-1]:.2f} 小时")
print(f"剪切后总时间: {len(Wake_bool_trimmed)/(freq*3600):.2f} 小时")
print(f"剪切后Wake比例: {percentages_trimmed[0]:.1f}%")
print(f"剪切后NREM比例: {percentages_trimmed[1]:.1f}%")
print(f"剪切后REM比例: {percentages_trimmed[2]:.1f}%")

In [ ]:
# 代码块 6: 剪切数据并计算详细统计
# 计算统计
total_time_hours = len(Wake_bool_trimmed) / (freq * 3600)

# 计算各阶段比例
wake_time = np.sum(Wake_bool_trimmed) / (freq * 3600)  # 小时
nrem_time = np.sum(NREM_bool_trimmed) / (freq * 3600)  # 小时
rem_time = np.sum(REM_bool_trimmed) / (freq * 3600)    # 小时

wake_percentage = wake_time / total_time_hours * 100
nrem_percentage = nrem_time / total_time_hours * 100
rem_percentage = rem_time / total_time_hours * 100

# 计算Wake周期数（睡眠碎片化指标）
# 找到Wake阶段的开始和结束
wake_diff = np.diff(Wake_bool_trimmed.astype(int))
wake_starts = np.where(wake_diff == 1)[0]
wake_ends = np.where(wake_diff == -1)[0]

# 如果数据以Wake开始，添加开始点
if Wake_bool_trimmed[0]:
    wake_starts = np.insert(wake_starts, 0, 0)

# 如果数据以Wake结束，添加结束点
if Wake_bool_trimmed[-1]:
    wake_ends = np.append(wake_ends, len(Wake_bool_trimmed)-1)

# Wake周期数
wake_episodes = len(wake_starts)

# 平均每小时Wake周期数
wake_episodes_per_hour = wake_episodes / total_time_hours

# 打印详细统计
print(f"=== {current_mouse_folder} 剪切后统计 ===")
print(f"开始剪切时间: {start_cut_time_hours} 小时")
if end_cut_time_hours is not None:
    print(f"结束剪切时间: {end_cut_time_hours} 小时")
else:
    print(f"结束剪切时间: 无 (使用数据末尾)")
print(f"总分析时间: {total_time_hours:.2f} 小时")
print(f"Wake时间: {wake_time:.2f} 小时 ({wake_percentage:.1f}%)")
print(f"NREM时间: {nrem_time:.2f} 小时 ({nrem_percentage:.1f}%)")
print(f"REM时间: {rem_time:.2f} 小时 ({rem_percentage:.1f}%)")
print(f"Wake周期数: {wake_episodes}")
print(f"平均每小时Wake周期数: {wake_episodes_per_hour:.2f}")

# 创建结果字典
results = {
    'folder': current_mouse_folder,
    'start_cut_time_hours': start_cut_time_hours,
    'end_cut_time_hours': end_cut_time_hours,
    'total_time_hours': total_time_hours,
    'wake_time_hours': wake_time,
    'nrem_time_hours': nrem_time,
    'rem_time_hours': rem_time,
    'wake_percentage': wake_percentage,
    'nrem_percentage': nrem_percentage,
    'rem_percentage': rem_percentage,
    'wake_episodes': wake_episodes,
    'wake_episodes_per_hour': wake_episodes_per_hour
}

In [ ]:
# 保存剪切后的分类结果
# 保存剪切后的分类结果 - 需要将布尔数组转换回索引列表
Wake_index_trimmed = np.where(Wake_bool_trimmed)[0].tolist()
REM_index_trimmed = np.where(REM_bool_trimmed)[0].tolist()
NREM_index_trimmed = np.where(NREM_bool_trimmed)[0].tolist()

trimmed_classification = {
    "Wake_index": Wake_index_trimmed,
    "REM_index": REM_index_trimmed,
    "NREM_index": NREM_index_trimmed,
    "freq": freq,
    "start_cut_time_hours": start_cut_time_hours,
    "end_cut_time_hours": end_cut_time_hours
}

# 确保输出目录存在
output_dir = os.path.join(filePath, current_mouse_folder, 'EEG_EMG')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

trimmed_json_path = os.path.join(output_dir, 'EEG_EMG_classfication_trimmed.json')
with open(trimmed_json_path, 'w') as f:
    json.dump(trimmed_classification, f)

print(f"剪切后的分类结果已保存到: {trimmed_json_path}")